## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ace_tools_open as tools; 

## Fetching Dataset

In [2]:
call_data = pd.read_csv("C:\\Users\\91954\\Downloads\\callsf0d4f5a.csv")
sentiment_stats = pd.read_csv("C:\\Users\\91954\\Downloads\\sentiment_statisticscc1e57a.csv")
test_data = pd.read_csv("C:\\Users\\91954\\Downloads\\testbc7185d.csv")
reason_data = pd.read_csv("C:\\Users\\91954\\Downloads\\reason18315ff.csv")
customer_data = pd.read_csv("C:\\Users\\91954\\Downloads\\customers2afd6ea.csv")

In [3]:
print(call_data)

          call_id  customer_id  agent_id call_start_datetime  \
0      4667960400   2033123310    963118     7/31/2024 23:56   
1      1122072124   8186702651    519057       8/1/2024 0:03   
2      6834291559   2416856629    158319     7/31/2024 23:59   
3      2266439882   1154544516    488324       8/1/2024 0:05   
4      1211603231   5214456437    721730       8/1/2024 0:04   
...           ...          ...       ...                 ...   
71805  1563273072   8022667294    413148     8/31/2024 23:48   
71806  8865997781   4474867021    980156     8/31/2024 23:55   
71807  8019240181   9762042472    616988     8/31/2024 23:52   
71808  8332067080   8095620266    161354     8/31/2024 23:53   
71809  8210720833   8014388758    547592     8/31/2024 23:49   

      agent_assigned_datetime call_end_datetime  \
0               8/1/2024 0:03     8/1/2024 0:34   
1               8/1/2024 0:06     8/1/2024 0:18   
2               8/1/2024 0:07     8/1/2024 0:26   
3               8/1/2024 0:

###  Convert relevant datetime columns to datetime objects for calculations


In [4]:
call_data['call_start_datetime'] = pd.to_datetime(call_data['call_start_datetime'])
call_data['agent_assigned_datetime'] = pd.to_datetime(call_data['agent_assigned_datetime'])
call_data['call_end_datetime'] = pd.to_datetime(call_data['call_end_datetime'])


### Calculate AHT (Average Handle Time) and AST (Average Speed to Answer)


In [5]:
call_data['AHT'] = (call_data['call_end_datetime'] - call_data['agent_assigned_datetime']).dt.total_seconds()
call_data['AST'] = (call_data['agent_assigned_datetime'] - call_data['call_start_datetime']).dt.total_seconds()


### Merging Dataset

In [6]:

merged_data = call_data.merge(sentiment_stats, on='call_id', how='left') \
                       .merge(reason_data, on='call_id', how='left')


In [7]:
merged_data = merged_data.merge(customer_data, on='customer_id', how='left')


In [8]:
tools.display_dataframe_to_user(name="Call Center Merged Data", dataframe=merged_data)

merged_data.head()

Call Center Merged Data


,call_id,customer_id,agent_id_x,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,AHT,AST,agent_id_y,agent_tone,customer_tone,average_sentiment,silence_percent_average,primary_call_reason,customer_name,elite_level_code
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,1860.0,420.0,963118,neutral,angry,-0.04,0.39,Voluntary Cancel,Matthew Foster,4.0
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,720.0,180.0,519057,calm,neutral,0.02,0.35,Booking,Tammy Walters,NaN
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,1140.0,480.0,158319,neutral,polite,-0.13,0.32,IRROPS,Jeffery Dixon,NaN
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,420.0,300.0,488324,neutral,frustrated,-0.20,0.20,Upgrade,David Wilkins,2.0
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,540.0,600.0,721730,neutral,polite,-0.05,0.35,Seating,Elizabeth Daniels,0.0


### Grouping data by primary_call_reason and calculating mean AHT and AST for each reason


In [9]:
aht_ast_by_reason = merged_data.groupby('primary_call_reason').agg({
    'AHT': 'mean',
    'AST': 'mean',
    'call_id': 'count'  # To find most and least frequent call reasons
}).reset_index()


###  Sorting by count of calls (to get most and least frequent reasons)


In [10]:
aht_ast_by_reason = aht_ast_by_reason.sort_values(by='call_id', ascending=False)


###  Most frequent and least frequent call reasons


In [11]:
most_frequent_reason = aht_ast_by_reason.iloc[0]
least_frequent_reason = aht_ast_by_reason.iloc[-1]


### Calculate percentage difference in AHT between most frequent and least frequent reason


In [12]:
aht_diff_percentage = (most_frequent_reason['AHT'] - least_frequent_reason['AHT']) / least_frequent_reason['AHT'] * 100


In [13]:
result = {
    "Most Frequent Call Reason": most_frequent_reason['primary_call_reason'],
    "Least Frequent Call Reason": least_frequent_reason['primary_call_reason'],
    "AHT for Most Frequent Reason": most_frequent_reason['AHT'],
    "AHT for Least Frequent Reason": least_frequent_reason['AHT'],
    "Percentage Difference in AHT": aht_diff_percentage
}



In [14]:
result

{'Most Frequent Call Reason': 'IRROPS',
 'Least Frequent Call Reason': 'Unaccompanied Minor  ',
 'AHT for Most Frequent Reason': 785.4913073447193,
 'AHT for Least Frequent Reason': 180.0,
 'Percentage Difference in AHT': 336.38405963595517}

###  Group data by primary call reason and check for recurring reasons


In [15]:
reason_frequency = merged_data['primary_call_reason'].value_counts().reset_index()
reason_frequency.columns = ['primary_call_reason', 'frequency']


###  Display the top 10 most frequent call reasons to identify potential IVR improvements


In [16]:
top_call_reasons = reason_frequency.head(10)


In [17]:
top_call_reasons

,primary_call_reason,frequency
0,IRROPS,13057
1,Voluntary Change,10291
2,Seating,6223
3,Mileage Plus,5487
4,Post-Flight,3869
5,Communications,3779
6,Products and Services,2792
7,Upgrade,2682
8,Baggage,2616
9,Booking,2589
